In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from snowflake.connector.pandas_tools import write_pandas
from sqlalchemy import create_engine
import snowflake.connector

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (14.0.2), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


In [2]:
import snowflake.connector

In [3]:
## Set options to view all columns
pd.set_option('display.max_columns', None)

In [4]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'TABLES'

In [5]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
)

In [6]:
cursor = conn.cursor()

In [120]:
cursor.execute('SELECT * FROM TEAMS')
team_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_names = pd.DataFrame(team_rows, columns=column_names)

In [121]:
cursor.execute('SELECT * FROM TEAM_MISC_STATS')
misc_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_misc = pd.DataFrame(misc_rows, columns=column_names)

In [122]:
cursor.execute('SELECT * FROM TEAM_STANDARD_STATS')
standard_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_standard = pd.DataFrame(standard_rows, columns=column_names)

In [123]:
cursor.execute('SELECT * FROM TEAM_ATTACKING_STATS')
attacking_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_attacking = pd.DataFrame(attacking_rows, columns=column_names)

In [124]:
cursor.execute('SELECT * FROM TEAM_DEFENDING_STATS')
defending_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_defending = pd.DataFrame(defending_rows, columns=column_names)

In [125]:
cursor.execute('SELECT * FROM COMPETITIONS')
competition_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df_competitions = pd.DataFrame(competition_rows, columns=column_names)

In [126]:
team_misc = team_misc.merge(df_competitions[['COMPETITION','COMPETITION_ACRONYM','SEASON']], 
                                      on=['COMPETITION','SEASON'], how='left')

In [127]:
team_misc = team_misc.merge(team_names, on='TEAM_FBREF_ID', how='left')

In [128]:
team_misc = team_misc.merge(team_standard, on=['TEAM_FBREF_ID', 'SEASON', 'COMPETITION'], how='left')

In [130]:
team_misc['AERIAL DUELS WON RATIO (%)'] = team_misc['AERIALS_WON']*100/(team_misc['AERIALS_WON'] + 
                                                                team_misc['AERIALS_LOST'])
team_misc['AERIAL DUELS ATTEMPTED PER GAME'] = (team_misc['AERIALS_WON'] + 
                                           team_misc['AERIALS_LOST'])/team_misc['MATCHES_PLAYED']

In [131]:
x_min_aerial = (team_misc['AERIAL DUELS WON RATIO (%)'].min() // 5)*5
x_max_aerial = (team_misc['AERIAL DUELS WON RATIO (%)'].max()//5)*5 + 5
y_min_aerial = (team_misc['AERIAL DUELS ATTEMPTED PER GAME'].min()//5)*5
y_max_aerial = (team_misc['AERIAL DUELS ATTEMPTED PER GAME'].max()//5)*5 + 5

In [132]:
(x_min_aerial, x_max_aerial, y_min_aerial, y_max_aerial)

(40.0, 60.0, 10.0, 45.0)

In [107]:
## Old limits: (45.0, 60.0, 20.0, 35.0)

In [134]:
team_goal_output = team_standard.merge(team_defending[['TEAM_FBREF_ID', 'SEASON', 'COMPETITION', 'XG_AGAINST']], 
                                       on=['TEAM_FBREF_ID', 'SEASON', 'COMPETITION'], how='left')

In [135]:
team_goal_output = team_goal_output.merge(df_competitions[['COMPETITION','COMPETITION_ACRONYM','SEASON']], 
                                      on=['COMPETITION','SEASON'], how='left')

In [136]:
team_goal_output = team_goal_output.merge(team_attacking[['TEAM_FBREF_ID', 'SEASON', 'COMPETITION','NPXG']], 
                                          on=['TEAM_FBREF_ID', 'SEASON', 'COMPETITION'], how='left')

In [137]:
team_goal_output['EXPECTED GOALS AGAINST PER GAME'] = team_goal_output['XG_AGAINST']/team_goal_output['MATCHES_PLAYED']

In [138]:
team_goal_output['NON PENALTY EXPECTED GOALS PER GAME'] = team_goal_output['NPXG']/team_goal_output['MATCHES_PLAYED']

In [139]:
team_goal_output = team_goal_output.merge(team_names, on='TEAM_FBREF_ID', how='left')

In [140]:
x_min_goal_output = 0.45
x_max_goal_output = 2.60
y_min_goal_output = 0.5
y_max_goal_output = 2.40

In [142]:
def create_FM_team_scatter_chart(df, x_axis_label, y_axis_label, img_size, x_min, x_max, y_min, y_max, bottom_left_label, 
                                 bottom_right_label, top_left_label, top_right_label, bl_color, br_color, tl_color, tr_color):
    fig = go.Figure()
    x_axis_mean_val = df[x_axis_label].mean()
    y_axis_mean_val = df[y_axis_label].mean()

    # Add the scatter plot points
    for index, row in df.iterrows():
        fig.add_trace(go.Scatter(
            x=[row[x_axis_label]],
            y=[row[y_axis_label]],
            mode='markers',
            text=row["TEAM_NAME"],
            marker=dict(
                size=1,  # Set a fixed size or normalize as you prefer
                opacity=0
            ),
            hoverinfo='text',
        ))

        # Add team logo as a layout_image
        fig.add_layout_image(
            dict(
                source=row["TEAM_LOGO_URL"],
                x=row[x_axis_label],
                y=row[y_axis_label],
                xref="x",
                yref="y",
                sizex=5,  # Adjust the size of the image here
                sizey=5,  # Adjust the size of the image here
                xanchor="center",
                yanchor="middle"
            )
        )

    # Update axes and layout as necessary
    # ...
        
    fig.update_xaxes(range=[x_min, x_max], title=x_axis_label)
    fig.update_yaxes(range=[y_min, y_max], title=y_axis_label)

    fig.add_shape(
        type='line',
        x0=x_axis_mean_val, y0=fig.layout.yaxis.range[0],  # start of the line
        x1=x_axis_mean_val, y1=fig.layout.yaxis.range[1],  # end of the line
        line=dict(color='White', width=3),
        layer='below'
    )

    # Add a horizontal line at the mean aerials attempted per game
    fig.add_shape(
        type='line',
        x0=fig.layout.xaxis.range[0], y0=y_axis_mean_val,  # start of the line
        x1=fig.layout.xaxis.range[1], y1=y_axis_mean_val,  # end of the line
        line=dict(color='White', width=3),
        layer='below'
    )

    fig.update_layout(
        width=900,
        height=900,
        showlegend=False,
        paper_bgcolor='rgb(70, 70, 70)',
        plot_bgcolor='rgb(70, 70, 70)',
        font=dict(
                family="Roboto, sans-serif",  # Specify the font family
                size=15,                     # Specify the font size
                color="white"                # Specify the font color
            ),
        hoverlabel=dict(
                bgcolor="rgba(20, 20, 20, 0.8)",
                font_family="Roboto, sans-serif"),
        images= [dict(
            source= row["TEAM_LOGO_URL"],
            xref="x",
            yref="y",
            x= row[x_axis_label],
            y= row[y_axis_label],
            sizex=img_size,  # The size of the image in x axis units
            sizey=img_size,  # The size of the image in y axis units
            sizing="contain",
            layer="above") for index, row in df.iterrows()]
    )

    fig.update_xaxes(
        showline=True,  # Show the axis line
        linewidth=2,  # Width of the axis line
        linecolor='white',  # Color of the axis line
        gridcolor='rgba(0,0,0,0)',  # Set grid line color to transparent
        tickfont=dict(color='white')  # Set the color of the axis ticks (numbers)
    )

    fig.update_yaxes(
        showline=True,
        linewidth=2,
        linecolor='white',
        gridcolor='rgba(0,0,0,0)',
        tickfont=dict(color='white')
    )

    fig.add_annotation(text=bottom_left_label,
                    xref="paper", yref="paper",
                    x=0, y=0,  # Bottom left corner
                    showarrow=False,
                    font=dict(size=14, color=bl_color, family="Roboto, sans-serif"),
                    align="left")

    fig.add_annotation(text=top_left_label,
                    xref="paper", yref="paper",
                    x=0, y=1,  # Top left corner
                    showarrow=False,
                    font=dict(size=14, color=tl_color, family="Roboto, sans-serif"),
                    align="left")

    fig.add_annotation(text=top_right_label,
                    xref="paper", yref="paper",
                    x=1, y=1,  # Top right corner
                    showarrow=False,
                    font=dict(size=14, color=tr_color, family="Roboto, sans-serif"),
                    align="right")

    fig.add_annotation(text=bottom_right_label,
                    xref="paper", yref="paper",
                    x=1, y=0,  # Bottom right corner
                    showarrow=False,
                    font=dict(size=14, color=br_color, family="Roboto, sans-serif"),
                    align="right")

    fig.show()

In [143]:
filtered_team_misc = team_misc[team_misc['SEASON'] == 2122]
filtered_team_misc = filtered_team_misc[filtered_team_misc['COMPETITION_ACRONYM'] == 'EPL']
filtered_team_goal_output = team_goal_output[team_goal_output['SEASON'] == 2122]
filtered_team_goal_output = filtered_team_goal_output[filtered_team_goal_output['COMPETITION_ACRONYM'] == 'EPL']

In [146]:
create_FM_team_scatter_chart(filtered_team_misc, 'AERIAL DUELS WON RATIO (%)', 'AERIAL DUELS ATTEMPTED PER GAME', 1.5, x_min_aerial, 
                             x_max_aerial, y_min_aerial, y_max_aerial, "Fewer Duels<br>Poor Dueling", "Fewer Duels<br>Strong Dueling",
                             "Lots of Duels<br>Poor Dueling", "Lots of Duels<br>Strong Dueling", "red", "orange", "orange", "green")

In [83]:
create_FM_team_scatter_chart(filtered_team_goal_output, 'EXPECTED GOALS AGAINST PER GAME', 'NON PENALTY EXPECTED GOALS PER GAME', 
                             0.1, x_min_goal_output, x_max_goal_output, y_min_goal_output, y_max_goal_output, 
                             "Low non-penalty expected goals<br>Strong Defending", 
                             "Low non-penalty expected goals<br>Poor Defending",
                             "High non-penalty expected goals<br>Strong Defending", 
                             "High non-penalty expected goals<br>Poor Defending", "orange", "red", "green", "orange")